# for simplicity ,we load dataset from keras

In [1]:
from keras.datasets import mnist
import shinnosuke
from shinnosuke.utils.Preprocess import to_categorical
from shinnosuke.layers.FC import Dense
from shinnosuke.layers.Dropout import Dropout
from shinnosuke.models import Sequential
import cupy as cp

Using TensorFlow backend.


In [10]:
batch_size = 512
num_classes = 10
epochs = 1

In [11]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Because we are going to use shinnosuke-gpu,so we convert np array to cp array

In [12]:
x_train=cp.asarray(x_train)
y_train=cp.asarray(y_train)
x_test=cp.asarray(x_test)
y_test=cp.asarray(y_test)

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [13]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [14]:
print('x_train shape:',x_train.shape)
print('y_train shape:',y_train.shape)
print('x_test shape:',x_test.shape)
print('y_test shape:',y_test.shape)

x_train shape: (60000, 784)
y_train shape: (60000, 10)
x_test shape: (10000, 784)
y_test shape: (10000, 10)


In [15]:
model = Sequential()
model.add(Dense(512, activation='relu', n_in=784))
model.add(Dropout(0.8))  #this's keep probability,whereas in Keras,it's drop probability
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(num_classes, activation='softmax'))


model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop'
)

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test)
print('Test loss:', score[1])
print('Test accuracy:', score[0])


 Epoch[1/1]
60000/60000 [==============================>] -52s -444ms/batch -batch_loss: 0.0912 -batch_acc: 0.9583 -val_loss: 0.1310 -val_acc: 0.9621
Test loss: 0.1310330716515285
Test accuracy: 0.9621


# Then we tested on keras-gpu

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

batch_size = 512
num_classes = 10
epochs = 1

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))  #drop probability
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 8s 131us/step - loss: 0.3802 - acc: 0.8801 - val_loss: 0.1485 - val_acc: 0.9547
Test loss: 0.1485417663320899
Test accuracy: 0.9547


# Keras-gpu is much faster than shinnosuke-gpu,that may due to  Keras-gpu is written by CUDA and most operations are conducted on GPU directly,while shinnosuke-gpu is written by cupy(a python library),so there exists a lot of data exchanges between GPU and memory,which remarkably slow down the speed.I will try to solve this problem in the future.
# By the way,shinnosuke get almost the same performance compares to Keras in this dataset.